# Big Data - Project

## TODO TOPIC

### Dragan Postolovsky
### Wojciech Taisner

TODO

A 1-page summary of litterature review on studies investigating stackoverflow data

TODO

An enumeration of "interesting" investigations directions for the 10-year dataset of stack overflow (from 09/2008 to 09/2018)

### What were the questions?

TODO

A report with analysis results (including visualisation, and statistics) of the 10 years data. These results are the fruit of the investigations projected earlier

In [1]:
#files
base = "stackoverflow.com-"

badges = "Badges"
comments = "Comments"
posthistory = "PostHistory"
postlinks = "PostLinks"
posts = "Posts"
tags = "Tags"
users = "Users"
votes = "Votes"

all_files = [badges, comments, postlinks, posts, tags, users, votes] # posthistory,
all_files

['Badges', 'Comments', 'PostLinks', 'Posts', 'Tags', 'Users', 'Votes']

In [2]:
# import dask bag
import dask.bag as db
# import BeatuifulSoup
from bs4 import BeautifulSoup

# read tags bag, select rows only
tags_bag = db.read_text(tags+".tar.xz", compression="xz").str.strip()
tags_bag = tags_bag.filter(lambda x: x.find("row") >= 0)

In [3]:
# non complex parsing (ex badges, tags, etc)
def xml_row_to_tuple(row):
    soup = BeautifulSoup(row, "xml")
    ret = []
    for a, v in soup.row.attrs.items():
        if v.isdigit():
            v = int(v) #integers take less memory :D
        ret.append(v)
    return tuple(ret);
def xml_headers_to_tuple(row):
    soup = BeautifulSoup(row, "xml")
    ret = []
    for a, v in soup.row.attrs.items():
        ret.append(a)
    return tuple(ret);

In [4]:
for row in tags_bag.take(4):
    print(row);
    print(xml_row_to_tuple(row))
    #soup = BeautifulSoup(row, "xml")
    #print(soup.row['Id'], soup.row['TagName'], soup.row['Count'])

<row Id="1" TagName=".net" Count="276857" ExcerptPostId="3624959" WikiPostId="3607476" />
(1, '.net', 276857, 3624959, 3607476)
<row Id="2" TagName="html" Count="789778" ExcerptPostId="3673183" WikiPostId="3673182" />
(2, 'html', 789778, 3673183, 3673182)
<row Id="3" TagName="javascript" Count="1723731" ExcerptPostId="3624960" WikiPostId="3607052" />
(3, 'javascript', 1723731, 3624960, 3607052)
<row Id="4" TagName="css" Count="563816" ExcerptPostId="3644670" WikiPostId="3644669" />
(4, 'css', 563816, 3644670, 3644669)


In [5]:
tags_bag.take(4) # return tuple
test = tags_bag.map(xml_row_to_tuple).compute()
test[0]
#carefull, takes a while

(1, '.net', 276857, 3624959, 3607476)

In [6]:
header = tags_bag.map(xml_headers_to_tuple).take(1)[0]
header

('Id', 'TagName', 'Count', 'ExcerptPostId', 'WikiPostId')

In [7]:
print(type(tags_bag))
print(type(test))
#tags_bag.map(xml_row_to_tuple).take(1)

<class 'dask.bag.core.Bag'>
<class 'list'>


In [8]:
# refiew structure of all files
# takes a while
extension=".tar.xz"
compression="xz"
for filename in all_files:
    print(filename)
    temp_bag = db.read_text(filename+extension, compression=compression).str.strip()
    temp_bag = temp_bag.filter(lambda x: x.find("row") >= 0)
    th = temp_bag.map(xml_headers_to_tuple).take(1)[0]
    print(th)
    #too long for showcase only
    #t = temp_bag.map(xml_row_to_tuple).compute()
    #print(t[0:4])
    tp = list(temp_bag.map(xml_row_to_tuple).take(4))
    print(tp)
    

Badges
('Id', 'UserId', 'Name', 'Date', 'Class', 'TagBased')
[(82946, 3718, 'Teacher', '2008-09-15T08:55:03.923', 3, 'False'), (82947, 994, 'Teacher', '2008-09-15T08:55:03.957', 3, 'False'), (82949, 3893, 'Teacher', '2008-09-15T08:55:03.957', 3, 'False'), (82950, 4591, 'Teacher', '2008-09-15T08:55:03.957', 3, 'False')]
Comments
('Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId')
[(2, 35314, 8, "Yeah, I didn't believe it until I created a console app - but good lord!  Why would they give you the rope to hang yourself!  I hated that about VB.NET - the OrElse and AndAlso keywords!", '2008-09-06T08:09:52.330', 3), (4, 35195, 0, "I don't see an accepted answer now, I wonder how that got unaccepted. Incidentally, I would have marked an accepted answer based on the answers available at the time. Also, accepted doesn't mean Best :)", '2008-09-06T08:42:16.980', 380), (9, 47239, 0, 'Jonathan: Wow!  Thank you for all of that, you did an amazing amount of work!', '2008-09-06T12:26:30.060',